In [ ]:
#import required modules
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
 
#Create spark configuration object
conf = SparkConf()
conf.setMaster("local").setAppName("My app")
 
#Create spark context and sparksession
sc = SparkContext.getOrCreate(conf=conf)
spark = SparkSession(sc)

#set variable to be used to connect the database
user = ""
password  = ""
host = ""
port = ""
database = "admitware"
# database = "Inquiry"
# # database = 'FWNEUCASE'

# user = ""
# password = ""
# host = ""
# port = ""
# database = "Admissions"

# %set_env PYSPARK_SUBMIT_ARGS=--packages=org.apache.hadoop:hadoop-aws:2.7.3 pyspark-shell

In [ ]:
table = "INFORMATION_SCHEMA.TABLES"
# table = "Application"
# table = "Student"
# table = "DBFieldName"
# table = "Inquiry"
 
#read table data into a spark dataframe
jdbcDF = spark.read.format("jdbc") \
    .option("url", f"jdbc:sqlserver://{host}:{port};databaseName={database};") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .load()
 
#show the data loaded into dataframe
jdbcDF.show()

In [ ]:
jdbcUrl=f"jdbc:sqlserver://{host}:{port};database={database}"

connectionProperties = {
  "user" : user,
  "password" : password,
  "driver" : "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

# jdbc_query = "select * from INFORMATION_SCHEMA.TABLES where TABLE_TYPE = 'BASE TABLE'"
jdbc_query = "select * from INFORMATION_SCHEMA.columns WHERE column_NAME LIKE '%admissionsdataid%'"
# jdbc_query = "select * from commonapp where First_name = 'Kaiyan'"
jdbc_table = "(" + jdbc_query + ") base_tables_alias"
partition_column = 'TABLE_NAME'

df = spark.read.jdbc( \
        url=jdbcUrl, \
        table=jdbc_table, \
                     
#         column=partition_column, \ # maximal number of concurrent JDBC connections
#         lowerBound=1, \
#         upperBound=100000, \
#         numPartitions=100, \
                     
#         predicates = ["TABLE_TYPE = 'BASE TABLE'" ], \
        numPartitions = 1, \
        properties=connectionProperties \
    )
# df.show()
# df.show(200)
# display(df)
# display(df.limit(20))
pd_df = df.limit(4000).toPandas().sort_values(by=['TABLE_NAME'])
# display(pd_df.head(1000).tail(300))
display(pd_df.head(3000))


In [ ]:
jdbcUrl=f"jdbc:sqlserver://{host}:{port};database={database}"

connectionProperties = {
  "user" : user,
  "password" : password,
  "driver" : "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

# query_table = 'Application'

# possible candidates...
# query_table = 'Inquiry'
# query_table = 'INQCase'
# query_table = 'SPECCase'
# query_table = 'NEUCase'

# query_table = 'StudentDocHistory'
# query_table = "GeoMarket"

###################

query_table = "NEUCASE"

jdbc_query = f"select top(100) * from {query_table} where NUID = '001387092'"
# jdbc_query = f"select top(100) * from {query_table} where admissionsdataid = 'N0071283103193' order by TermYear desc"

jdbc_table = "(" + jdbc_query + ") base_tables_alias"
# partition_column = 'TABLE_NAME'

df = spark.read.jdbc( \
        url=jdbcUrl, \
        table=jdbc_table, \
                     
#         column=partition_column, \ # maximal number of concurrent JDBC connections
#         lowerBound=1, \
#         upperBound=100000, \
#         numPartitions=100, \
                     
#         predicates = ["TABLE_TYPE != 'BASE TABLE'", "TABLE_TYPE = 'BASE TABLE'" ], \
#         numPartitions = 1, \
        properties=connectionProperties \
    )
# df.show(200)
# display(df)
# display(df.limit(20))

pd_df = df.limit(2000).toPandas()
# display(pd_df[['ACT_Composite_Score', 'IB_Subject_total_number', 'IELTS_Overall_Band_Score', 'SAT_CR_Score', 'SAT_Math_Score', 'SAT_Subject_total_number']])
# print(pd_df)
display(pd_df)

# print(pd_df)

for c in pd_df.columns:
    print(c)



In [ ]:
import os
f"jdbc:sqlserver://{os.environ.get('ADMITWARE_HOST')}:{os.environ.get('ADMITWARE_PORT')};database={os.environ.get('ADMITWARE_DATABASE')}"

In [ ]:
from pyspark import SparkConf, SparkContext, sql

# sc = SparkContext('local', 'Test')
# spark = sql.SparkSession \
# .builder \
# .appName("TEST") \
# .getOrCreate()

# sc._jsc.hadoopConfiguration().set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
# sc._jsc.hadoopConfiguration().set("fs.s3a.multipart.uploads.enabled", "true")

sc._jsc.hadoopConfiguration().set("fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.AssumedRoleCredentialProvider")
sc._jsc.hadoopConfiguration().set("fs.s3a.assumed.role.arn", "arn:aws:iam::209299999084:role/ep-test-compute-node-role")

sql_context = sql.SQLContext(sc, spark)
filename = 'admitware/test.parquet'
s3_uri = 's3a://nu-data-lake-test/{}'.format(filename)
print(s3_uri)
df = sql_context.createDataFrame([('1', '4'), ('2', '5'), ('3', '6')], ["A", "B"])
df.write.parquet(s3_uri)
# df.write.parquet("s3a://nu-data-lake-test/admitware/test.parquet",mode="overwrite")
